# Food recommendation system

## load dataset and tools

In [5]:
import pandas as pd
import numpy as np

In [9]:
df=pd.read_csv('/Users/user/Downloads/recipe_dataset.csv')

In [11]:
df.head()

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,15,36,1,42,21,81,2,"['pork belly', 'smoked paprika', 'kosher salt'..."
1,1,240488,"Pork Loin, Apples, and Sauerkraut",4.76,https://images.media-allrecipes.com/userphotos...,29,19,18,10,73,33,104,41,"['sauerkraut drained', 'Granny Smith apples sl..."
2,2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,17,36,2,48,24,31,4,"['chicken wings', 'sprigs rosemary', 'head gar..."
3,3,87211,Chicken Pesto Paninis,4.62,https://images.media-allrecipes.com/userphotos...,163,32,45,20,65,20,43,18,"['focaccia bread quartered', 'prepared basil p..."
4,4,245714,Potato Bacon Pizza,4.50,https://images.media-allrecipes.com/userphotos...,2,8,12,5,14,7,8,3,"['red potatoes', 'strips bacon', 'Sauce:', 'he..."


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48735 entries, 0 to 48734
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        48735 non-null  int64  
 1   recipe_id         48735 non-null  int64  
 2   recipe_name       48735 non-null  object 
 3   aver_rate         48735 non-null  float64
 4   image_url         48735 non-null  object 
 5   review_nums       48735 non-null  int64  
 6   calories          48735 non-null  int64  
 7   fat               48735 non-null  int64  
 8   carbohydrates     48735 non-null  int64  
 9   protein           48735 non-null  int64  
 10  cholesterol       48735 non-null  int64  
 11  sodium            48735 non-null  int64  
 12  fiber             48735 non-null  int64  
 13  ingredients_list  48735 non-null  object 
dtypes: float64(1), int64(10), object(3)
memory usage: 5.2+ MB


In [17]:
df.shape

(48735, 14)

In [30]:
df.isnull().sum()

Unnamed: 0          0
recipe_id           0
recipe_name         0
aver_rate           0
image_url           0
review_nums         0
calories            0
fat                 0
carbohydrates       0
protein             0
cholesterol         0
sodium              0
fiber               0
ingredients_list    0
dtype: int64

In [19]:
df['recipe_name']

0                           Homemade Bacon
1        Pork Loin, Apples, and Sauerkraut
2         Foolproof Rosemary Chicken Wings
3                    Chicken Pesto Paninis
4                       Potato Bacon Pizza
                       ...                
48730               Grateful Dead Cocktail
48731          Cheese Filling For Pastries
48732                       Peach Smoothie
48733                  Double Dare Peaches
48734           All-Purpose Marinara Sauce
Name: recipe_name, Length: 48735, dtype: object

In [21]:
df['ingredients_list']

0        ['pork belly', 'smoked paprika', 'kosher salt'...
1        ['sauerkraut drained', 'Granny Smith apples sl...
2        ['chicken wings', 'sprigs rosemary', 'head gar...
3        ['focaccia bread quartered', 'prepared basil p...
4        ['red potatoes', 'strips bacon', 'Sauce:', 'he...
                               ...                        
48730    ['fluid ounce tequila', 'fluid ounce vodka', '...
48731    ['raisins', 'brandy', 'cream cheese', 'white s...
48732    ['sliced peaches drained', 'scoops vanilla ice...
48733    ['butter', 'habanero peppers', 'fresh peaches'...
48734    ['olive oil', 'bulb garlic', 'tomatoes chopped...
Name: ingredients_list, Length: 48735, dtype: object

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
vectorizer = TfidfVectorizer()
x_ingredients=vectorizer.fit_transform(df['ingredients_list'])

In [1]:
x_ingredients.toarray().shape

NameError: name 'x_ingredients' is not defined

### Scalling

In [32]:
from sklearn.preprocessing import StandardScaler

In [34]:
scaler = StandardScaler()
x=scaler.fit_transform(df[['calories','fat','carbohydrates','protein','cholesterol','sodium','fiber']])

In [59]:
x.shape

(48735, 7)

In [63]:
X_combined=np.hstack([x,x_ingredients.toarray()])

In [42]:
X_combined

array([[-0.1317045 ,  0.46001924, -1.15482863, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.23857551, -0.33625589, -0.01920347, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05343551,  0.46001924, -1.02864806, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.77969453, -0.82286847, -0.27156462, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.33114552,  0.32730672,  0.73787996, ...,  0.        ,
         0.        ,  0.        ],
       [-1.33511455, -0.9998185 , -1.02864806, ...,  0.        ,
         0.        ,  0.        ]])

## train the model

In [46]:
from sklearn.neighbors import NearestNeighbors

In [48]:
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn.fit(X_combined)

NearestNeighbors(metric='euclidean', n_neighbors=3)

# Function to Recommend Recipes

In [51]:
def recommend_recipes(inputs):
    scaled=scaler.transform(inputs[:7])
    transformed=vectorizer.transform(inputs[7:])
    input_combined=np.hstack(scaled,transformed.toarray())
    distances,indices=knn.kneighbors(input_combined)
    recommendation=df.iloc[indices[0]]
    return recommendation[['recipe_name', 'ingredients_list', 'image_url']]

In [57]:
# Example Input
input_features = [15, 36, 1, 42, 21, 81, 2, 'pork belly, smoked paprika, kosher salt'].values.reshape(1,-1)
recommendations = recommend_recipes(input_features)
recommendations

AttributeError: 'list' object has no attribute 'values'